In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.15.0'

### 케라스의 순환 층 : SimpleRNN

- 시퀀스 배치를 처리 : (batch_size, timesteps, input_features) 크기의 입력을 받음
- 두 가지 모드로 실행: return_sequences로 선택
  - 각 타임스텝의 출력을 모은 전체 시퀀스를 반환: 크기가 (batch_size, timesteps, output_features) 3D 텐서, : return_sequences=True
  - 입력 시퀀스에 대한 마지막 출력만 반환 : 크기가 (batch_size, output_features)인 2D 텐서


#### 예제1. SimpleRNN을 사용하여 마지막 타임스텝의 출력 얻기

In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Embedding

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=32))
model.add(SimpleRNN(units=32))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
Total params: 322080 (1.23 MB)
Trainable params: 322080 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### 예제2. SimpleRNN을 사용하여 전체 상태 시퀀스 반환

In [4]:
inputs = keras.Input(shape=(None, ))
x = Embedding(10000,32)(inputs)
outputs = SimpleRNN(32, return_sequences=True)(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
Total params: 322080 (1.23 MB)
Trainable params: 322080 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### 예제3. 여러 개의 순환층을 사용한 RNN 모델
- 중간층들이 전체 출력 시퀀스를 반환하도록 설정해야 함

In [5]:
inputs = keras.Input(shape=(None, ))
x = Embedding(10000,32)(inputs)
x= SimpleRNN(32, return_sequences=True)(x)
x= SimpleRNN(32, return_sequences=True)(x)
x= SimpleRNN(32, return_sequences=True)(x)
x= SimpleRNN(32, return_sequences=True)(x)
outputs = SimpleRNN(32)(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 32)                2080

---

### IMDB 영화리뷰 분류를 위한 RNN : SimpleRNN 층 사용

- Embedding층과 SimpleRNN층, Dense층으로 구성한 모델 사용

####  imdb 데이터 로딩 및 데이터 전처리

In [6]:
from keras.datasets import imdb
from keras.utils import pad_sequences

max_features = 10000
maxlen = 500
batch_size = 32

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train.shape, x_test.shape

17464789/17464789 [==============================] - 0s 0us/step


((25000,), (25000,))

- 제로 패딩(zero padding)

In [7]:
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [8]:
x_train.shape, x_test.shape

((25000, 500), (25000, 500))

#### IMDB 분류를 위한 간단한 순환 신경망 정의 및 컴파일

In [9]:
inputs = keras.Input(shape=(None, ))
x = Embedding(max_features,batch_size)(inputs)
x = SimpleRNN(batch_size)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 322113 (1.23 MB)
Trainable params: 322113 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.compile(optimizer= keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

#### 순환신경망 훈련

In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
chekpt = ModelCheckpoint('model/imdb_best_SimpleRNN.h5')
stopping = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=50, batch_size= 128,
                    validation_split=0.2, callbacks=[chekpt, stopping])

Epoch 1/50
157/157 [==============================] - 26s 152ms/step - loss: 0.6945 - accuracy: 0.5090 - val_loss: 0.6914 - val_accuracy: 0.5278
Epoch 2/50
  1/157 [..............................] - ETA: 19s - loss: 0.6819 - accuracy: 0.6016

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


 32/157 [=====>........................] - ETA: 21s - loss: 0.6880 - accuracy: 0.5540

KeyboardInterrupt: 

####  순환 신경망 모델의 성능(훈련, 검증의 손실과 정확도) 시각화

### 모델 테스트

----

### 연습문제

1. SimpleRNN 층을 두 개로 둔 모델로 학습하고 예측성능 평가
   - max_features = 10000
   - maxlen = 500
   - batch_size = 32

2. SimpleRNN 층을 세 개로 둔 모델로 학습하고 예측성능 평가
    - max_features = 10000
    - maxlen = 500
    - batch_size = 32

3. SimpleRNN 층의 batch_size를 8, 16, 32로 변경하여 학습한 경우 비교 평가

-----